
# End-to-End Indicator ML Strategy (BTCUSDT)

Blueprint notebook to transform the provided feature-rich CSVs into a clean feature layer, label future returns, train a baseline ML model, and backtest simple probability-to-position mappings. The main focus is the 1H dataset, with comparison against 4H and 1D.



## 1. Imports & Paths
Load core dependencies and configure the file locations for each timeframe. The 1H file is treated as the primary source while keeping 4H and 1D available for quick comparisons.


In [ ]:
import os
import math
import subprocess
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_columns", None)


def discover_candidate_data_dirs() -> List[Path]:
    """Enumerate plausible data directories, including env overrides and repo root."""
    candidates: List[Path] = []
    seen = set()

    def add(path_like):
        path = Path(path_like).resolve()
        if path in seen:
            return
        seen.add(path)
        candidates.append(path)

    env_dir = os.getenv("DATA_DIR")
    if env_dir:
        add(env_dir)

    cwd = Path.cwd().resolve()
    add(cwd / "data")
    add(cwd / "notebooks" / "data")
    for parent in cwd.parents:
        add(parent / "data")
        add(parent / "notebooks" / "data")

    try:
        repo_root = Path(
            subprocess.check_output(["git", "rev-parse", "--show-toplevel"], text=True).strip()
        )
        add(repo_root / "data")
        add(repo_root / "notebooks" / "data")
    except Exception:
        pass

    return [p for p in candidates if p.exists()]


def resolve_data_path(filename: str) -> Path:
    data_dirs = discover_candidate_data_dirs()
    attempts = []
    for base in data_dirs:
        candidate = base / filename
        attempts.append(candidate)
        if candidate.exists():
            return candidate
    raise FileNotFoundError(
        f"Could not locate data file '{filename}'. Checked: {attempts or ['<no data dirs found>']}. "
        "Set DATA_DIR env var or place the file under one of the listed data directories."
    )


DATA_FILES = {
    "1h": "BINANCE_BTCUSDT.P, 60.csv",
    "4h": "BINANCE_BTCUSDT.P, 240.csv",
    "1d": "BINANCE_BTCUSDT.P, 1D.csv",
}
DATA_PATHS = {tf: resolve_data_path(filename) for tf, filename in DATA_FILES.items()}


In [ ]:

def compute_adx(high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14) -> pd.Series:
    prev_close = close.shift(1)
    tr = pd.concat([
        high - low,
        (high - prev_close).abs(),
        (low - prev_close).abs(),
    ], axis=1).max(axis=1)

    plus_dm = (high - high.shift(1)).clip(lower=0)
    minus_dm = (low.shift(1) - low).clip(lower=0)
    plus_dm = plus_dm.where(plus_dm > minus_dm, 0.0)
    minus_dm = minus_dm.where(minus_dm > plus_dm, 0.0)

    tr_smooth = tr.ewm(alpha=1/period, adjust=False).mean()
    plus_smooth = plus_dm.ewm(alpha=1/period, adjust=False).mean()
    minus_smooth = minus_dm.ewm(alpha=1/period, adjust=False).mean()

    plus_di = 100 * (plus_smooth / tr_smooth).replace([np.inf, -np.inf], np.nan)
    minus_di = 100 * (minus_smooth / tr_smooth).replace([np.inf, -np.inf], np.nan)
    dx = (100 * (plus_di - minus_di).abs() / (plus_di + minus_di)).replace([np.inf, -np.inf], np.nan)
    adx = dx.ewm(alpha=1/period, adjust=False).mean()
    return adx


def add_strategy_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['ret_1'] = df['close'].pct_change()
    df['bb_percent'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    df['bb_percent'] = df['bb_percent'].clip(-5, 5)
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_basis']

    df['donchian_high_20'] = df['high'].rolling(20).max()
    df['donchian_low_20'] = df['low'].rolling(20).min()
    df['vol_rel_50'] = df['volume'] / df['volume'].rolling(50).mean()

    df['ret_20'] = df['close'].pct_change(20)
    df['ret_50'] = df['close'].pct_change(50)
    df['vol_20'] = df['ret_1'].rolling(20).std()

    df['adx'] = compute_adx(df['high'], df['low'], df['close'])
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill()
    return df


def build_feature_layer(path: Path) -> pd.DataFrame:
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(
            f"Data file not found: {path}. Checked candidate data dirs: {discover_candidate_data_dirs()}."
        )

    df = pd.read_csv(path)
    df = rename_and_filter_columns(df)

    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values('time').drop_duplicates(subset=['time'])
    df = df.set_index('time')
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.ffill().bfill()
    df = df.reset_index()

    df = add_strategy_features(df)
    return df


In [ ]:

RENAME_MAP = {
    'time': 'time',
    'open': 'open',
    'high': 'high',
    'low': 'low',
    'close': 'close',
    'Volume': 'volume',
    'VWAP': 'vwap',
    'Upper Band #1': 'bb1_upper',
    'Lower Band #1': 'bb1_lower',
    'Basis': 'bb_basis',
    'Upper': 'bb_upper',
    'Lower': 'bb_lower',
    'Up Trend': 'trend_up',
    'Down Trend': 'trend_down',
    'EMA': 'ema_fast',
    'EMA.1': 'ema_slow',
    'Conversion Line': 'ichi_conv',
    'Base Line': 'ichi_base',
    'Lagging Span': 'ichi_lag',
    'Leading Span A': 'ichi_span_a',
    'Leading Span B': 'ichi_span_b',
    'Upper.1': 'channel_upper',
    'Average': 'channel_mid',
    'Lower.1': 'channel_lower',
    'RSI': 'rsi',
    'RSI-based MA': 'rsi_ma',
    'Regular Bullish': 'div_bull',
    'Regular Bullish Label': 'div_bull_label',
    'Regular Bearish': 'div_bear',
    'Regular Bearish Label': 'div_bear_label',
    'Histogram': 'macd_hist',
    'MACD': 'macd',
    'Signal': 'macd_signal',
    'ATR': 'atr',
    'K': 'stoch_k',
    'D': 'stoch_d',
    '%R': 'williams_r',
}

DROP_PATTERNS = [
    'PlotCandle',
]
DROP_EXACT = {'Plot', 'Plot.1', 'Plot.2', 'div_bull_label', 'div_bear_label'}



## 3. Data Loading & Feature Layer Construction
Helper functions to load a timeframe, clean columns, fill gaps, and add a few helper features (returns, Bollinger %B, Bollinger width). The result is the **feature layer** ready for labeling and modeling.


In [ ]:
def rename_and_filter_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns=RENAME_MAP)
    keep_cols = []
    for col in df.columns:
        if any(pattern in col for pattern in DROP_PATTERNS):
            continue
        if col in DROP_EXACT:
            continue
        keep_cols.append(col)
    return df[keep_cols]



## 4. Labeling (Binary Up/Down)
Create a forward return label for a chosen horizon. By default we use 4 bars ahead. `y = 1` when the future return is positive, else `0`.


In [ ]:

def add_labels(df: pd.DataFrame, horizon: int = 4) -> pd.DataFrame:
    future_price = df['close'].shift(-horizon)
    df['future_return'] = future_price / df['close'] - 1
    df['y'] = (df['future_return'] > 0).astype(int)
    df = df.dropna(subset=['future_return'])
    return df



## 5. Train/Validation/Test Split (Time-Based)
Use chronological splits (70/15/15). No shuffling is applied to respect temporal order.


In [ ]:

@dataclass
class DatasetSplit:
    X_train: pd.DataFrame
    X_val: pd.DataFrame
    X_test: pd.DataFrame
    y_train: pd.Series
    y_val: pd.Series
    y_test: pd.Series
    future_val: pd.Series
    future_test: pd.Series
    train_df: pd.DataFrame
    val_df: pd.DataFrame
    test_df: pd.DataFrame


def time_based_split(df: pd.DataFrame, feature_cols: List[str]) -> DatasetSplit:
    n = len(df)
    train_end = int(n * 0.7)
    val_end = int(n * 0.85)

    train = df.iloc[:train_end]
    val = df.iloc[train_end:val_end]
    test = df.iloc[val_end:]

    X_train, y_train = train[feature_cols], train['y']
    X_val, y_val = val[feature_cols], val['y']
    X_test, y_test = test[feature_cols], test['y']

    return DatasetSplit(
        X_train=X_train,
        X_val=X_val,
        X_test=X_test,
        y_train=y_train,
        y_val=y_val,
        y_test=y_test,
        future_val=val['future_return'],
        future_test=test['future_return'],
        train_df=train,
        val_df=val,
        test_df=test,
    )



## 6. Modeling + Threshold Search
Train a LightGBM classifier inside a scikit-learn pipeline (imputer + scaler). Validation Sharpe on the forward-return horizon is used to pick the best probability threshold for entries.


In [ ]:
def compute_sharpe(returns: np.ndarray, periods_per_year: int = 365) -> float:
    ret = np.array(returns)
    if ret.std() == 0:
        return 0.0
    return (ret.mean() * periods_per_year) / (ret.std() * math.sqrt(periods_per_year))


def evaluate_threshold(probs: np.ndarray, future_returns: pd.Series, candidate_thr: List[float]):
    best_sharpe, best_thr = -np.inf, None
    for thr in candidate_thr:
        positions = (probs > thr).astype(int)
        strat_ret = positions * future_returns.values
        sharpe = compute_sharpe(strat_ret)
        if sharpe > best_sharpe:
            best_sharpe, best_thr = sharpe, thr
    return best_thr, best_sharpe


def train_model(split: DatasetSplit, feature_cols: List[str]):
    imputer = SimpleImputer(strategy="median")
    scaler = StandardScaler()

    imputer.fit(split.X_train)
    scaler.fit(imputer.transform(split.X_train))

    def transform(df: pd.DataFrame) -> pd.DataFrame:
        arr = imputer.transform(df)
        arr = scaler.transform(arr)
        return pd.DataFrame(arr, columns=feature_cols, index=df.index)

    X_train = transform(split.X_train)
    X_val = transform(split.X_val)
    X_test = transform(split.X_test)

    model = lgb.LGBMClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary",
    )
    model.fit(X_train, split.y_train)

    val_probs = model.predict_proba(X_val)[:, 1]
    candidate_thr = [0.5, 0.55, 0.6, 0.65, 0.7]
    best_thr, best_sharpe = evaluate_threshold(val_probs, split.future_val, candidate_thr)

    test_probs = model.predict_proba(X_test)[:, 1]
    positions_test = (test_probs > best_thr).astype(int)
    strat_ret_test = positions_test * split.future_test.values

    metrics = {
        "val_accuracy": accuracy_score(split.y_val, (val_probs > 0.5).astype(int)),
        "test_accuracy": accuracy_score(split.y_test, (test_probs > 0.5).astype(int)),
        "val_sharpe@best_thr": best_sharpe,
        "test_sharpe@best_thr": compute_sharpe(strat_ret_test),
        "threshold": best_thr,
    }

    reports = {
        "val_report": classification_report(split.y_val, (val_probs > 0.5).astype(int), digits=3),
        "test_report": classification_report(split.y_test, (test_probs > 0.5).astype(int), digits=3),
    }

    processed = {"train": X_train, "val": X_val, "test": X_test}

    return model, metrics, reports, val_probs, test_probs, processed



## 7. End-to-End Runner (Per Timeframe)
Combine all steps for a given timeframe and collect summary metrics. The primary focus is `1h`, with side-by-side stats for `4h` and `1d`.


In [ ]:

def run_timeframe(key: str, horizon: int = 4):
    df = build_feature_layer(DATA_PATHS[key])
    df = add_labels(df, horizon=horizon)

    feature_cols = [
        col for col in df.columns
        if col not in {'time', 'y', 'future_return'}
        and not col.startswith('div_')  # drop visual divergence labels
    ]

    split = time_based_split(df, feature_cols)
    model, metrics, reports, val_probs, test_probs, processed = train_model(split, feature_cols)

    val_probs = pd.Series(val_probs, index=split.X_val.index, name='prob_up')
    test_probs = pd.Series(test_probs, index=split.X_test.index, name='prob_up')

    summary = {
        'timeframe': key,
        'rows': len(df),
        **metrics,
    }
    return {
        'summary': summary,
        'reports': reports,
        'model': model,
        'feature_cols': feature_cols,
        'split': split,
        'processed_features': processed,
        'val_probs': val_probs,
        'test_probs': test_probs,
        'df': df,
    }

results = {tf: run_timeframe(tf) for tf in DATA_PATHS}
summary_df = pd.DataFrame([results[tf]['summary'] for tf in DATA_PATHS])
summary_df



### Classification Reports (quick check)
View precision/recall/F1 for each timeframe to understand base discrimination performance.


In [ ]:

for tf, res in results.items():
    print(f"=== {tf} ===")
    print(res['reports']['test_report'])



## 8. SHAP Explainability (Feature Importance)
Use SHAP to see which indicators drive the `prob_up` predictions. Run on a subset for speed.


In [ ]:
# Uncomment if shap is not available in your environment
# %pip install shap

import io
from pathlib import Path
import matplotlib
matplotlib.use('Agg')
import warnings
from IPython.display import Image, display

# Suppress noisy LightGBM / SHAP output format warnings
warnings.filterwarnings(
    'ignore',
    message='LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray',
    category=UserWarning,
)

IMG_DIR = Path('outputs/shap')
IMG_DIR.mkdir(parents=True, exist_ok=True)

try:
    import shap
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    print("shap is not installed in this environment. Install with `pip install shap` to view explainability plots.")
else:
    shap.initjs()

    primary = results['1h']
    X_explain = primary['processed_features']['test'].sample(
        n=min(1000, len(primary['processed_features']['test'])), random_state=42
    )
    explainer = shap.TreeExplainer(primary['model'])
    shap_values = explainer.shap_values(X_explain)
    if isinstance(shap_values, list):
        shap_values_class1 = shap_values[1] if len(shap_values) > 1 else shap_values[0]
    else:
        shap_values_class1 = shap_values

    def _save_and_display(fig, filename: str):
        out_path = IMG_DIR / filename
        buf = io.BytesIO()
        fig.savefig(out_path, bbox_inches='tight', dpi=150)
        fig.savefig(buf, format='png', bbox_inches='tight', dpi=150)
        buf.seek(0)
        display(Image(data=buf.read(), format='png', embed=True))

    try:
        shap.summary_plot(shap_values_class1, X_explain, feature_names=primary['feature_cols'], show=False)
        _save_and_display(plt.gcf(), 'shap_beeswarm_1h.png')

        shap.summary_plot(
            shap_values_class1,
            X_explain,
            feature_names=primary['feature_cols'],
            plot_type='bar',
            show=False,
        )
        _save_and_display(plt.gcf(), 'shap_bar_1h.png')
    except Exception as exc:
        print(f"SHAP plotting skipped: {exc}")
    finally:
        plt.close('all')



## 9. Next Steps & Risk Layer
- Apply strategy-specific filters (reversal/trend/volatility) using the `prob_up` outputs and indicators (RSI, bb_percent, macd_hist, EMA stacks, ATR).
- Add risk management: ATR-based SL/TP, max risk per trade, exposure caps.
- Extend to multi-asset or orderbook factors when data is available.



## 10. Strategy pack implementation (10 strategies)
Below are executable, backtest-ready implementations for the ten strategy templates you shared. They consume the `prob_up` outputs produced above plus the indicator features already in the dataset. Strategies that require unavailable data (funding/orderbook/multi-asset) will still emit a status row so you see coverage gaps.


In [ ]:

from typing import Callable


def strategy_metrics(positions: pd.Series, future_returns: pd.Series, name: str, note: str = "") -> Dict[str, float]:
    strat_ret = positions * future_returns
    metrics = {
        'strategy': name,
        'trades': int((positions != 0).sum()),
        'exposure': float((positions != 0).mean()),
        'total_return': float(strat_ret.sum()),
        'sharpe': compute_sharpe(strat_ret),
        'win_rate': float((strat_ret > 0).mean()) if len(strat_ret) else 0.0,
        'note': note or 'ok',
    }
    return metrics


def run_strategy_pack(result: Dict) -> pd.DataFrame:
    df = result['split'].test_df.copy().reset_index(drop=True)
    df['prob_up'] = result['test_probs'].reset_index(drop=True)

    def reversal(df):
        positions = pd.Series(0, index=df.index, dtype=float)
        sideways = (df['adx'] < 20) | (df['bb_width'] < df['bb_width'].rolling(50).median())
        long = sideways & (df['prob_up'] > 0.6) & (df['rsi'] < 30) & (df['bb_percent'] < 0.1)
        short = sideways & (df['prob_up'] < 0.4) & (df['rsi'] > 70) & (df['bb_percent'] > 0.9)
        positions[long] = 1
        positions[short] = -1
        return positions

    def trend_follow(df):
        positions = pd.Series(0, index=df.index, dtype=float)
        in_trend = df['adx'] > 25
        bull = in_trend & (df['ema_fast'] > df['ema_slow'])
        bear = in_trend & (df['ema_fast'] < df['ema_slow'])
        positions[bull & (df['prob_up'] > 0.55)] = 1
        positions[bear & (df['prob_up'] < 0.45)] = -1
        return positions

    def vol_mean_reversion(df):
        positions = pd.Series(0, index=df.index, dtype=float)
        extreme_oversold = (df['bb_percent'] < 0.05) & (df['rsi'] < 25)
        high_vol = df['bb_width'] > df['bb_width'].rolling(100).median()
        positions[extreme_oversold & high_vol & (df['prob_up'] > 0.65)] = 1
        return positions

    def orderbook_momentum(df):
        positions = pd.Series(0, index=df.index, dtype=float)
        if 'vol_rel_50' not in df.columns:
            return positions, 'requires volume proxy'
        imbalance_proxy = df['vol_rel_50'] > 1.5
        positions[imbalance_proxy & (df['prob_up'] > 0.7)] = 1
        positions[df['prob_up'] < 0.3] = -1
        return positions, 'using volume spike as imbalance proxy'

    def breakout(df):
        positions = pd.Series(0, index=df.index, dtype=float)
        vol_thresh = df['bb_width'].rolling(50).median()
        breaking_up = (df['close'] > df['donchian_high_20']) & (df['bb_width'] > vol_thresh)
        volume_spike = df['vol_rel_50'] > 1.5
        positions[breaking_up & volume_spike & (df['prob_up'] > 0.6)] = 1
        return positions

    def ts_momentum(df):
        positions = pd.Series(0, index=df.index, dtype=float)
        mom = df['ret_20']
        high_mom = mom > mom.quantile(0.7)
        low_mom = mom < mom.quantile(0.3)
        positions[high_mom & (df['prob_up'] > 0.55)] = 1
        positions[low_mom & (df['prob_up'] < 0.45)] = -1
        return positions

    strategies: List[Tuple[str, Callable[[pd.DataFrame], Tuple[pd.Series, str]]]] = [
        ('reversal', lambda d: (reversal(d), 'ok')),
        ('trend_follow', lambda d: (trend_follow(d), 'ok')),
        ('vol_mean_reversion', lambda d: (vol_mean_reversion(d), 'ok')),
        ('orderbook_momentum', orderbook_momentum),
        ('breakout_vol_expansion', lambda d: (breakout(d), 'ok')),
        ('ts_momentum_vol_target', lambda d: (ts_momentum(d), 'ok')),
        ('funding_carry', lambda d: (pd.Series(0, index=d.index, dtype=float), 'requires funding/basis data')),
        ('cross_sectional_momentum', lambda d: (pd.Series(0, index=d.index, dtype=float), 'requires multi-asset data')),
        ('stat_arb_pairs', lambda d: (pd.Series(0, index=d.index, dtype=float), 'requires paired instruments')),
        ('multi_timeframe_ensemble_single', lambda d: (pd.Series(0, index=d.index, dtype=float), 'computed separately below')),
    ]

    rows = []
    for name, fn in strategies:
        positions, note = fn(df)
        rows.append(strategy_metrics(positions, df['future_return'], name, note))
    return pd.DataFrame(rows)


strategy_tables = {tf: run_strategy_pack(results[tf]) for tf in results}
strategy_tables['1h']



### Multi-timeframe ensemble (1H + 4H + 1D)
This cell aligns the three test splits with `merge_asof` (using 1H as the base), then applies the ensemble rule: 1H > 0.55, 4H > 0.60, 1D > 0.50. It reports backtest metrics on the aligned sample.


In [ ]:

base = results['1h']['split'].test_df[['time', 'future_return']].copy().reset_index(drop=True)
base['prob_1h'] = results['1h']['test_probs'].reset_index(drop=True)

four_h = results['4h']['split'].test_df[['time']].copy().reset_index(drop=True)
four_h['prob_4h'] = results['4h']['test_probs'].reset_index(drop=True)

one_d = results['1d']['split'].test_df[['time']].copy().reset_index(drop=True)
one_d['prob_1d'] = results['1d']['test_probs'].reset_index(drop=True)

aligned = pd.merge_asof(base.sort_values('time'), four_h.sort_values('time'), on='time', direction='backward')
aligned = pd.merge_asof(aligned.sort_values('time'), one_d.sort_values('time'), on='time', direction='backward')

positions = pd.Series(0, index=aligned.index, dtype=float)
cond = (
    (aligned['prob_1h'] > 0.55)
    & (aligned['prob_4h'] > 0.60)
    & (aligned['prob_1d'] > 0.50)
)
positions[cond] = 1
ensemble_metrics = strategy_metrics(positions, aligned['future_return'], 'multi_timeframe_ensemble', 'asof aligned')
ensemble_metrics


## 10. How to read the outputs

- **What the notebook gives you**
  - `summary_df` (cell 7) → quick table of rows per timeframe plus Sharpe-like metric and accuracy stats for train/val/test. Use this to spot if a timeframe is overfitting (train >> val/test) or too noisy (all metrics ~0.5).
  - Classification reports (cell 8) → precision/recall/F1 per class. Focus on `val` first; test should be close. If you see heavy imbalance, adjust thresholds or class weights.
  - SHAP plots (cell 10) saved to files and displayed inline → top features driving the 1h model. Beeswarm shows directionality (red = high value, blue = low value). Bar plot ranks mean(|SHAP|) importance.

- **How to check data/loading**
  - Cell 3 prints a clear error if a CSV is missing and shows the resolved `PROJECT_ROOT`. If you run elsewhere, set `DATA_ROOT` env var to your folder containing the `BINANCE_BTCUSDT.P, *.csv` files.
  - Ensure the `time` column parses correctly and the index is sorted; the helper already enforces this.

- **How to interpret SHAP**
  - Points on the **right** increase `prob_up`; on the **left** decrease it.
  - Red cluster on the left for `rsi` means overbought reduces the up-probability; blue on the right means oversold supports upside.
  - If a feature you expect (e.g., `trend_up` or `macd_hist`) is missing from top ranks, re-check column renaming or drop rules.

- **What to do next (practical checks)**
  - Re-run cells 7–10 after any data or feature change; keep an eye on validation Sharpe and SHAP ranking stability.
  - If a timeframe underperforms, tweak `horizon`, add/remove features, or raise the prob threshold in cell 6’s search grid.
  - Add simple fee/slippage assumptions in your backtest to see net performance.

- **Save artifacts**
  - Export trained models/scalers if you want to reuse them: `joblib.dump(model, 'outputs/lgbm_1h.pkl')` and keep the feature list from `primary['feature_cols']`.
  - The SHAP images are saved in the working directory; move them to `outputs/` for record-keeping if needed.



## Strategi referensi & cara uji (indikator + ML)

Gunakan tabel SHAP dan metrik validasi yang sudah ada untuk memilih strategi mana yang layak diujikan, lalu mapping ke logika berikut.

### Cara praktis menjalankan uji strategi
1. **Siapkan fitur wajib** per strategi di `feature_cols` (lihat daftar indikator di bawah). Jika ADX belum ada di CSV, hitung dulu dari OHLCV (bisa pakai `ta.trend.ADXIndicator`).
2. **Pakai `results['<tf>']`** yang sudah berisi `train/val/test` + `prob_up`. Tambahkan kolom rule-based pada dataframe hasil sebelum backtest kecil:
   ```python
   df_tf = results['1h']['data']  # gunakan kunci 4h/1d sesuai kebutuhan
   # contoh template entry/exit dengan prob_up + rule
   df_tf['position'] = 0
   long_mask = <rule_long>
   short_mask = <rule_short>
   df_tf.loc[long_mask, 'position'] = 1
   df_tf.loc[short_mask, 'position'] = -1
   ```
   Setelah itu jalankan evaluator sederhana (PnL / Sharpe) seperti di sel validasi/test.
3. **Tuning threshold**: iterasi `prob_up` cutoff (mis. 0.55–0.7 untuk long, 0.3–0.45 untuk short) dan filter indikator yang relevan, catat Sharpe/MaxDD per kombinasi.
4. **Bandingkan lintas timeframe**: jalankan rule sama untuk `1h/4h/1d`, lalu pilih setup dengan trade frequency, Sharpe, dan drawdown paling sehat.

### Ringkasan 10 strategi library
- **(1) Reversal (sideways)** — butuh: `rsi`, `macd_hist`, `vwap`, `volume`, `bb_percent`, `bb_width`, `adx` (optional). Entry: `adx < 20`, long jika `prob_up > 0.6`, `rsi < 30`, `bb_percent < 0.1`; short jika `prob_up < 0.4`, `rsi > 70`, `bb_percent > 0.9`.
- **(2) Trend-Following** — butuh: `ema_20`, `ema_50`, `ema_200`, `adx`, `volume`. Entry long: `adx > 25` & EMA berurutan naik & `prob_up > 0.55`; entry short: `adx > 25`, EMA berurutan turun, `prob_up < 0.45`; else `position = 0`.
- **(3) Volatility Mean-Reversion** — butuh: `bb_percent`, `bb_width`, `atr`, `rsi`. Entry long: `bb_percent < 0.05`, `rsi < 25`, `bb_width` di atas median rolling 100, `prob_up > 0.65`; exit ide: `rsi > 50` atau `bb_percent > 0.5`.
- **(4) Orderbook Momentum (advanced)** — butuh data orderbook: `imbalance`, `bid/ask volume spike`. Entry long: `imbalance > 0.6` & `prob_up > 0.7`; short: `prob_up < 0.3`. Pastikan efek latensi/slippage diperhitungkan di backtest.
- **(5) Multi-Timeframe Ensemble** — gunakan `prob_up` per TF (`pred_1h`, `pred_4h`, `pred_1d`). Entry = 1 jika `pred_1h > 0.55` & `pred_4h > 0.60` & `pred_1d > 0.50`. Untuk versi stacking, fit meta `LogisticRegression` pada kolom prediksi.
- **(6) Breakout & Vol Expansion** — butuh: `donchian_high/low` atau highest/lowest N-bar, `bb_width` atau `atr`, `volume`. Entry long: `close > donchian_high`, `bb_width > thr_vol`, volume > 1.5× MA50, `prob_up > 0.6`.
- **(7) Time-Series Momentum + Vol Targeting** — butuh: rolling return (20/50), rolling std, `atr`. Entry long jika `mom_20 > quantile(0.7)` & `prob_up > 0.55`; leverage = `target_vol / realized_vol` (clip maks 3x), posisi = `position * leverage`.
- **(8) Funding/Basis Carry (perp futures)** — butuh: `funding`, basis (perp−spot)/spot, tren EMA. Short perp jika `funding` di quantile 0.8, tren tidak turun kuat, `prob_up < 0.45`; long perp jika funding sangat negatif & `prob_up > 0.55`.
- **(9) Cross-Sectional Momentum (multi-coin)** — butuh data multi-simbol. Hitung `mom_20` per coin, rank per waktu; long top-N, optional short bottom-N. Bisa ganti ranking berdasarkan `prob_up` model per coin.
- **(10) Stat Arb / Pairs Trading** — butuh pasangan cointegrated (spread, z-score). Entry long A/short B jika `z < -2` & `prob_revert > 0.6`; sebaliknya untuk short A/long B.

### Cara membaca hasil
- **Feature importance (SHAP)**: gunakan top fitur untuk memutuskan filter strategi mana yang paling relevan; mis. jika `bb_percent` dan `rsi` dominan, prioritaskan strategi (1)/(3).
- **Metrik validasi**: pilih strategi yang menjaga Sharpe/Sortino dan MaxDD setelah menerapkan rule di atas, bukan hanya AUC/accuracy.
- **Artefak simpanan**: setelah menemukan kombinasi terbaik, simpan model, scaler, dan plot ke `outputs/indikator-ml-strategy/` agar dapat direpro.
